## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import warnings
import re

# PyTorch
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler, Dataset

# Hugging Face Transformers
from transformers import CamembertTokenizer, CamembertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import TrainerCallback
from transformers import Trainer
from transformers import TrainingArguments


# Scikit-learn packages for modeling and evaluation
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split


#!pip install GPUtil
'''
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import string
import re

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'\nimport torch\nfrom GPUtil import showUtilization as gpu_usage\nfrom numba import cuda\nimport string\nimport re\n\ndef free_gpu_cache():\n    print("Initial GPU Usage")\n    gpu_usage()\n\n    torch.cuda.empty_cache()\n\n    cuda.select_device(0)\n    cuda.close()\n    cuda.select_device(0)\n\n    print("GPU Usage after emptying the cache")\n    gpu_usage()\n\nfree_gpu_cache()\n'

### Data Loading

In [ ]:
labeled_df = pd.read_csv( "labeled_df.csv")
labeled_df = labeled_df[['label', 'id', 'text']]
labeled_df.head()
num_rows = len(labeled_df)
print(f"Number of rows: {num_rows}")

# Get unique values and their counts in the 'label' column
unique_labels = labeled_df['label'].unique()
print(f"Unique labels: {unique_labels}")

label_counts = labeled_df['label'].value_counts()
print("Counts of each label:")
print(label_counts)

duplicate_ids = labeled_df[labeled_df['id'].duplicated(keep=False)]

# Displaying the rows with duplicate IDs
print(duplicate_ids)


Number of rows: 339
Unique labels: [0 1]
Counts of each label:
1    174
0    165
Name: label, dtype: int64
     label          id                                               text
5        1  0BV7191EOS  Passer de 75% à 100% de couverture des cantine...
94       1  HAANZ1HGAQ  ----- Aemro Selassie (FMI) : " L’Afrique doit ...
120      1  N3BEDGSJZU  Bénin : l'intégralité du conseil des ministres...
123      0  NDBP5YOL02  Gestion des cantines scolaires au Bénin : Une ...
147      1  U02ORISCP5  Les glaciers du Kilimandjaro, sommet des défis...
304      1  0BV7191EOS  Passer de 75% à 100% de couverture des cantine...
317      1  NDBP5YOL02  Gestion des cantines scolaires au Bénin : Une ...
319      1  N3BEDGSJZU  Bénin : l'intégralité du conseil des ministres...
320      1  U02ORISCP5  Les glaciers du Kilimandjaro, sommet des défis...
324      1  HAANZ1HGAQ  ----- Aemro Selassie (FMI) : " L’Afrique doit ...


In [ ]:
# First, ensure that if 'NDBP5YOL02' is duplicated with different labels, keep the one with label 1
special_case = labeled_df[(labeled_df['id'] == 'NDBP5YOL02') & (labeled_df['label'] == 1)]
if not special_case.empty:
    # If the special case exists, remove all other 'NDBP5YOL02' entries
    labeled_df = labeled_df.drop(labeled_df[(labeled_df['id'] == 'NDBP5YOL02')].index)
    # Append the special case back to the dataframe
    labeled_df = pd.concat([labeled_df, special_case], ignore_index=True)

# Step 2: Remove all other duplicates, keeping the first occurrence
labeled_df = labeled_df.drop_duplicates(subset=['id'], keep='first')
print(labeled_df[(labeled_df['id'] == 'NDBP5YOL02') ])

     label          id                                               text
337      1  NDBP5YOL02  Gestion des cantines scolaires au Bénin : Une ...


In [ ]:
df = pd.read_csv( "/content/drive/My Drive/df.csv")

# Select only 'id' and 'text' columns
df= df[['id', 'text']]

# Add an empty 'label' column
df['label'] = np.nan

# Display the modified DataFrame to verify
df.head()

,id,text,label
0,HO8KNVZ6QF,Le plan d’autonomie est une “solution de compr...,NaN
1,N7HP3S8B9V,Un quatuor béninois pour arbitrer Mali U23 vs ...,NaN
2,ORG3BSXN7V,------------------- distinguée femme leader de...,NaN
3,ULGT4CHQHH,La CCI Bénin signe avec les CCI de Bahreïn et ...,NaN
4,2HTTXAR4Q8,RADARISTES EST TROP !\n\nOncle AGBAYA\nOn vous...,NaN


In [ ]:
# Get a list of unique IDs from the modified labeled_df
unique_ids_in_labeled_df = labeled_df['id'].unique()

# Remove rows from df that have IDs matching those in labeled_df
df = df[~df['id'].isin(unique_ids_in_labeled_df)]

## Data Cleaning / Preprocessing

In [ ]:
def clean_text(text):
    """
    Remove URLs and other unwanted patterns from the text.
    """
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Remove lines with dashes or similar patterns
    text = re.sub(r'-{2,}', '', text)

    # Remove emails or specific patterns (example)
    text = re.sub(r'\S*@\S*\s?', '', text)

    # Any additional cleaning steps can be added here

    return text



In [ ]:
# Apply the cleaning function to your DataFrame directly on the text column
labeled_df['text'] = labeled_df['text'].apply(clean_text)
df['text'] = df['text'].apply(clean_text)

display(labeled_df.head())
display(df.head())

label_counts = labeled_df['label'].value_counts()
print("Counts of each label:")
print(label_counts)



,label,id,text
0,0,02XWR02BCE,"Crise sanitaire, recrutements, succession… Les..."
1,0,03URLSTT7L,Transfert de l’énergie solaire depuis l’espace...
2,1,08XCWWYT57,Des céréales ukrainiennes vers l’Afrique : le ...
3,1,09746LR4F6,Les députés en séance plénière pour l'examen d...
4,1,09QJEPKIPQ,"L’ANSD relève une progression de 0, 8 % de l’i..."


,id,text,label
0,HO8KNVZ6QF,Le plan d’autonomie est une “solution de compr...,NaN
1,N7HP3S8B9V,Un quatuor béninois pour arbitrer Mali U23 vs ...,NaN
2,ORG3BSXN7V,distinguée femme leader de l’année 2022\n\nLa...,NaN
3,ULGT4CHQHH,La CCI Bénin signe avec les CCI de Bahreïn et ...,NaN
4,2HTTXAR4Q8,RADARISTES EST TROP !\n\nOncle AGBAYA\nOn vous...,NaN


Counts of each label:
1    170
0    164
Name: label, dtype: int64


In [ ]:

# Split the initial DataFrame into train+validation and test sets
train_val_df, test_df = train_test_split(
    labeled_df,
    test_size=0.2,  # 20% of the data goes to the test set
    stratify=labeled_df['label'],  # Stratify split by the 'label' column to maintain label distribution
    random_state=42  # For reproducibility
)

# Optionally, drop the 'id' column or any other columns not needed for training
train_val_df = train_val_df.drop(columns=['id'])
test_df = test_df.drop(columns=['id'])


# Further split the train_val_df into actual training and validation sets
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.25,  # Assuming you want 80/20 split for train/validation from the original 80% train_val set
    stratify=train_val_df['label'],  # Stratify by 'label' to maintain distribution
    random_state=42  # For reproducibility
)

# Display the first few rows of the train and test DataFrames to verify everything looks correct
display(train_df.head())
display(val_df.head())
display(test_df.head())

# Print the shapes of the new DataFrames to verify the split
print(f"Train set shape: {train_val_df['label'].value_counts()}")
print(f"Validation set shape: {val_df['label'].value_counts()}")
print(f"Test set shape: {test_df['label'].value_counts()}")

,label,text
151,0,Foire départementale sur l’énergie solaire : ...
268,0,Conseil des ministres au Bénin : confection de...
250,0,"Exploitation du sable : "" Nous avons atteint l..."
264,0,Commerce extérieur : baisse des exportations d...
288,1,Bénin : un programme national de développement...


,label,text
121,0,"Gabon - : "" doit comprendre le danger d’un n..."
161,0,Cancer du sein: une étude conseille ce type d’...
48,1,Le directeur général de l’ISRA appelle à ‘’pos...
111,0,"au sujet de la variole du singe: "" L’épidémie..."
175,1,Forum mondial sur la nutrition de l’enfant: e...


,label,text
298,1,[DOSSIER] Souveraineté alimentaire du Sénégal ...
65,0,(COVID-19) Bénin : allègement des mesures prév...
277,1,(Éclairage) Gel des prix des denrées au Sénéga...
188,0,Décès d’un jeune dans une piscine à Hillacondj...
205,0,TARéS & TERREUR - 24 Heures au Bénin\n\nOncle ...


Train set shape: 1    136
0    131
Name: label, dtype: int64
Validation set shape: 1    34
0    33
Name: label, dtype: int64
Test set shape: 1    34
0    33
Name: label, dtype: int64


## Model Initialization

### Camambert Tokenizer

In [ ]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

def tokenize_dataframe(df, tokenizer):
    return tokenizer(
        df['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=512,  # Adjust this as needed
        return_tensors="pt"  # Returns PyTorch tensors
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Tokenize each of the pre-split DataFrames
train_encodings = tokenize_dataframe(train_df, tokenizer)
val_encodings = tokenize_dataframe(val_df, tokenizer)
test_encodings = tokenize_dataframe(test_df, tokenizer)

# Convert labels to list for compatibility
train_labels = train_df['label'].tolist()
val_labels = val_df['label'].tolist()
test_labels = test_df['label'].tolist()

# Create datasets for each set
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# Verify the dataset splits by printing their lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Train dataset length: 200
Validation dataset length: 67
Test dataset length: 67


### initializing Parameters

In [ ]:
import wandb
# Initialize wandb
wandb.init(project="MLM_binary", entity="arisoy10")

sweep_config = {
    'method': 'bayes',  # or 'grid', 'random'
    'metric': {
      'name': 'eval_loss',
      'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 5e-4
        },
        'num_train_epochs': {
            'values': [4]
        },
        'per_device_train_batch_size': {
            'values': [8, 16, 32]
        }
        # Add other hyperparameters here
    }
}
sweep_id = wandb.sweep(sweep_config, project="TER_test")

Create sweep with ID: 3k77hog4
Sweep URL: https://wandb.ai/arisoy/TER_test/sweeps/3k77hog4


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import TrainingArguments, Trainer, TrainerCallback

'''
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }'''

# Define compute_metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [ ]:
#model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=2)  # Adjust num_labels as per your task
model = CamembertForSequenceClassification.from_pretrained("/content/drive/My Drive/MyModel", num_labels=2)
#!pip install trainer
import os
os.environ["WANDB_WATCH"] = "all"  # Automatically log gradients and model parameters

def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Get hyperparameters
        config = wandb.config

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=config.num_train_epochs,
            per_device_train_batch_size=config.per_device_train_batch_size,
            per_device_eval_batch_size=24,
            logging_dir='./logs',
            logging_steps=4,  # Assuming you want to log after each step for a small dataset
            evaluation_strategy="epoch",
            save_strategy='epoch',   # Evaluate at the end of each epoch
            learning_rate=3e-5,
            load_best_model_at_end=True,
            report_to="wandb",
            # Add warmup steps if you are using a scheduler that requires them
            warmup_steps=0,  # Example, adjust as needed
            # Make sure to add any additional arguments that you might need
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
        )

    # Start training
        trainer.train()

        # Evaluate the model
        metrics = trainer.evaluate()

            # Log both loss and accuracy to wandb
        wandb.log({"eval_loss": metrics["eval_loss"], "accuracy": metrics["eval_accuracy"]})


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/MyModel and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
wandb.agent(sweep_id, function=train, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: nuzcrecn with config:
wandb: 	learning_rate: 0.000380200125100912
wandb: 	num_train_epochs: 4
wandb: 	per_device_train_batch_size: 8


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.645600,0.635266,0.731343,0.763158,0.690476,0.852941
2,0.557800,0.519031,0.835821,0.835821,0.848485,0.823529
3,0.417700,0.462247,0.791045,0.800000,0.777778,0.823529
4,0.357100,0.449141,0.776119,0.782609,0.771429,0.794118


Checkpoint destination directory ./results/checkpoint-25 already exists and is non-empty. Saving will proceed but saved results may be invalid.
<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory ./results/checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory ./results/checkpoint-75 already exists and is non-empty. Savi

accuracy,▁
eval/accuracy,▁█▅▄▄
eval/f1,▁█▅▃▃
eval/loss,█▄▁▁▁
eval/precision,▁█▅▅▅
eval/recall,█▅▅▁▁
eval/runtime,█▁▁▁▁
eval/samples_per_second,▁██▇█
eval/steps_per_second,▁██▇█
eval_loss,▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇█████


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: koexgayv with config:
wandb: 	learning_rate: 0.00013956920221796826
wandb: 	num_train_epochs: 4
wandb: 	per_device_train_batch_size: 16
wandb: Currently logged in as: arisoy10 (arisoy). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.318800,0.440091,0.791045,0.794118,0.794118,0.794118
2,0.244500,0.435913,0.835821,0.830769,0.870968,0.794118


<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Create the confusion matrix
cm = confusion_matrix(test_labels, preds)

# Create a custom colormap
cmap = sns.diverging_palette(220, 20, as_cmap=True)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Log the custom colored confusion matrix to wandb
wandb.log({"Confusion Matrix": wandb.Image(plt)})
plt.close()

In [ ]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


# Apply softmax to the predictions to get probabilities
probabilities = softmax(predictions.predictions, axis=1)[:, 1]

# Now you can continue with calculating the ROC curve and plotting
fpr, tpr, thresholds = roc_curve(test_labels, probabilities)
roc_auc = auc(fpr, tpr)


# Create ROC curve plot
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

# Log the ROC curve plot
wandb.log({"ROC Curve": wandb.Image(plt)})
plt.close()


precision, recall, _ = precision_recall_curve(test_labels, probabilities)

# Create precision-recall curve plot
plt.figure()
plt.plot(recall, precision, color='blue', lw=lw, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="upper right")

# Log the precision-recall curve plot
wandb.log({"Precision-Recall Curve": wandb.Image(plt)})
plt.close()

## WandB and Training

In [ ]:
#!pip install wandb
import wandb
wandb.init(project="MLM-binary", entity="arisoy10")


eval/accuracy,▁███
eval/f1,▁▇██
eval/loss,█▅▂▁
eval/precision,▁███
eval/recall,█▁▆▆
eval/runtime,▄█▅▁
eval/samples_per_second,▅▁▄█
eval/steps_per_second,▅▁▄█
train/epoch,▁▂▃▅▆██
train/global_step,▁▂▃▅▆███
train/grad_norm,█▁


In [ ]:
trainer.train()  # Evaluate on the validation set

<ipython-input-35-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.678586,0.567164,0.701031,0.539683,1.000000
2,0.688700,0.656056,0.761194,0.800000,0.695652,0.941176
3,0.659800,0.628989,0.791045,0.815789,0.738095,0.911765
4,0.659800,0.615099,0.791045,0.815789,0.738095,0.911765


Checkpoint destination directory ./results/checkpoint-7 already exists and is non-empty. Saving will proceed but saved results may be invalid.
<ipython-input-35-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory ./results/checkpoint-14 already exists and is non-empty. Saving will proceed but saved results may be invalid.
<ipython-input-35-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory ./results/checkpoint-21 already exists and is non-empty. Savin

TrainOutput(global_step=28, training_loss=0.6622569050107684, metrics={'train_runtime': 673.9897, 'train_samples_per_second': 1.187, 'train_steps_per_second': 0.042, 'total_flos': 210488844288000.0, 'train_loss': 0.6622569050107684, 'epoch': 4.0})

<ipython-input-35-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
wandb.finish()

eval/accuracy,▁▇██
eval/f1,▁▇██
eval/loss,█▆▃▁
eval/precision,▁▇██
eval/recall,█▃▁▁
eval/runtime,█▁▃▃
eval/samples_per_second,▁█▆▆
eval/steps_per_second,▁█▆▆
train/epoch,▁▂▃▅▆██
train/global_step,▁▂▃▅▆████
train/grad_norm,█▁


### Evaluating

In [ ]:
# Step1
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Step 2: Extract actual labels from the validation dataset 'val_dataset'
actual_labels = [val_dataset[i]['labels'].item() for i in range(len(val_dataset))]

# Step 3: Identify indices of false positives
false_positives_indices = [i for i, (pred, actual) in enumerate(zip(predicted_labels, actual_labels)) if pred == 0 and actual == 1]

# Step 4: Decode and print false positives for review
for idx in false_positives_indices:
    # Assuming your dataset returns PyTorch tensors, use `.numpy()` to convert them for decoding
    input_ids = val_dataset[idx]['input_ids'].numpy()
    decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    print(f"False Positive Text at index {idx}: {decoded_text}")

False Positive Text at index 3: Agriculture : Les producteurs Sénégalais ont reçu un don de 25.000 tonnes d'engrais de l'OCP. C'est dans le cadre de la lutte contre l'insécurité alimentaire que l’Office chérifien du phosphate (OCP) du royaume du Maroc, a octroyé au Sénégal un don de 10. 000 tonnes d'engrais complexe et 15.000 tonnes d'engrais phosphaté, destiné aux petits producteurs sénégalais. La cérémonie officielle de remise d'engrais de ce don s'est tenue, ce Jeudi 20 Octobre aux entrepôts de Diamniadio en présence du ministre de l’agriculture, de l’équipement rural et de la souveraineté alimentaire,. À l'occasion de cette cérémonie, le ministre a exprimé sa joie sur la relation fraternelle qui existe entre le Royaume du Maroc et la République du Sénégal, une relation qui se traduit par une coopération étroite entre le ministère de l'agriculture du Sénégal et l'Office chérifien du phosphate à travers des conventions et des activités techniques relatives à la fertilité des sols et 

<ipython-input-35-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(test_dataset)

# Print the performance metrics
print("Test Performance:", test_results)

<ipython-input-9-2aee9a0a0f49>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Error: You must call wandb.init() before wandb.log()